## Data Colletion from Twitter

In [1]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
import itertools
import numpy as np
import re

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

In [2]:
stopwords = ["for", "on", "an", "a", "of", "and", "in", "the", "to", "from"]

def clean_tweet(tweet):
    if type(tweet) == np.float:
        return ""
    temp = tweet.lower()
    temp = re.sub("'", "", temp) # to avoid removing contractions in english
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    temp = re.sub(r'http\S+', '', temp)
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('\[.*?\]',' ', temp)
    temp = re.sub("[^a-z0-9]"," ", temp)
    temp = temp.split()
    temp = [w for w in temp if not w in stopwords]
    temp = " ".join(word for word in temp)
    return temp


def get_data(city, amount, distance = '20km'):
    df_city = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(f'near:"{city}" within:{distance}').get_items(), amount))[['date', 'content']]
    df_city['cleaned_content'] = [clean_tweet(i) for i in df_city['content']]
    df_city.to_csv(f"data/{city}.csv")
    return df_city

In [ ]:
%%time
get_data("Accra", 10000)
get_data("Dublin", 10000)
get_data("Jamaica", 10000)
get_data("London", 10000)
get_data("Melbourne", 10000)
get_data("New Delhi", 10000)
get_data("New York", 10000)
get_data("Singapore", 10000)
get_data("Manila", 10000)

CPU times: user 1min 11s, sys: 5.09 s, total: 1min 16s
Wall time: 58min 17s


'thank you thank you'

,date,content,cleaned_content
0,2022-05-20 08:29:17+00:00,@a100_rs 😂💔,
1,2022-05-20 08:29:16+00:00,@ManeJObaba 😁,
2,2022-05-20 08:29:12+00:00,Ban plastics and ban all mining for some years,ban plastics ban all mining some years
3,2022-05-20 08:29:12+00:00,@Polo_Capalot @pikaso_me please screenshot this,please screenshot this
4,2022-05-20 08:29:11+00:00,@stonebwoy ❤️,
...,...,...,...
9995,2022-05-20 01:56:31+00:00,@TrishiaNasa ebi like I wan take you away fro him 😂😂😂💔,ebi like i wan take you away fro him
9996,2022-05-20 01:56:30+00:00,@Ordinaryganaian Madness,madness
9997,2022-05-20 01:56:27+00:00,@FisicoImpuro Nuevamente \n@FisicoImbecil alias NALGA LOCA,nuevamente alias nalga loca
9998,2022-05-20 01:56:26+00:00,@AnfieldWatch On them!!!,them
